# 🎬 LTX-2 Video Generation

Generate videos with Lightricks' LTX-2 model (19B parameters).

**Requirements:**
- Google Colab Pro+ with A100 GPU (40GB) recommended
- Or Colab Pro with aggressive memory optimization

**Features:**
- Text-to-Video (T2V)
- Image-to-Video (I2V)
- Audio generation included

---

## 1️⃣ Check GPU

In [ ]:
# Check GPU availability and memory
import torch

if not torch.cuda.is_available():
    print("❌ No GPU found! Go to Runtime > Change runtime type > GPU")
else:
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"✅ GPU: {gpu_name}")
    print(f"✅ VRAM: {gpu_mem:.1f} GB")
    
    if gpu_mem < 20:
        print("\n⚠️ Warning: Less than 20GB VRAM. Generation may fail.")
        print("   Consider using Colab Pro+ with A100 GPU.")
    elif gpu_mem < 40:
        print("\n✅ Should work with FP8 optimization and smaller videos.")
    else:
        print("\n✅ Plenty of VRAM! Full quality generation possible.")

## 2️⃣ Install Dependencies

In [ ]:
# Install LTX-2 from GitHub
!pip install -q uv

# Clone repository
!git clone -q https://github.com/getelementbyiq/Anymore.git /content/LTX-2
%cd /content/LTX-2

# Install dependencies
!uv pip install --system -q -e packages/ltx-core -e packages/ltx-pipelines
!pip install -q huggingface_hub gradio

print("✅ Dependencies installed!")

## 3️⃣ Download Models

This will download ~32GB of models. Takes about 5-10 minutes.

In [ ]:
import os
from huggingface_hub import hf_hub_download, snapshot_download

MODEL_DIR = "/content/models"
os.makedirs(MODEL_DIR, exist_ok=True)

print("📥 Downloading LTX-2 models...")
print("   This takes 5-10 minutes on Colab.\n")

# Download main checkpoint (FP8 for memory efficiency)
print("1/4 Downloading main checkpoint (~19GB)...")
hf_hub_download(
    repo_id="Lightricks/LTX-2",
    filename="ltx-2-19b-distilled-fp8.safetensors",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)
print("   ✅ Checkpoint downloaded")

# Download spatial upsampler
print("2/4 Downloading spatial upsampler...")
hf_hub_download(
    repo_id="Lightricks/LTX-2",
    filename="ltx-2-spatial-upscaler-x2-1.0.safetensors",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)
print("   ✅ Upsampler downloaded")

# Download distilled LoRA
print("3/4 Downloading distilled LoRA...")
hf_hub_download(
    repo_id="Lightricks/LTX-2",
    filename="ltx-2-19b-distilled-lora-384.safetensors",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)
print("   ✅ LoRA downloaded")

# Download Gemma text encoder
print("4/4 Downloading Gemma text encoder (~12GB)...")
GEMMA_PATH = os.path.join(MODEL_DIR, "gemma-3-12b-it-qat-q4_0-unquantized")
snapshot_download(
    repo_id="google/gemma-3-12b-it-qat-q4_0-unquantized",
    local_dir=GEMMA_PATH,
    local_dir_use_symlinks=False,
)
print("   ✅ Gemma downloaded")

print("\n✅ All models downloaded!")

# Show disk usage
!du -sh /content/models/*

## 4️⃣ Load Pipeline

In [ ]:
import os
import torch

# Set memory optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Paths
MODEL_DIR = "/content/models"
CHECKPOINT_PATH = os.path.join(MODEL_DIR, "ltx-2-19b-distilled-fp8.safetensors")
UPSAMPLER_PATH = os.path.join(MODEL_DIR, "ltx-2-spatial-upscaler-x2-1.0.safetensors")
LORA_PATH = os.path.join(MODEL_DIR, "ltx-2-19b-distilled-lora-384.safetensors")
GEMMA_PATH = os.path.join(MODEL_DIR, "gemma-3-12b-it-qat-q4_0-unquantized")

print("🔄 Loading LTX-2 pipeline...")
print("   This takes 2-3 minutes.\n")

from ltx_core.loader import LTXV_LORA_COMFY_RENAMING_MAP, LoraPathStrengthAndSDOps
from ltx_pipelines.distilled import DistilledPipeline

# Configure distilled LoRA
distilled_lora = [
    LoraPathStrengthAndSDOps(
        LORA_PATH,
        0.8,
        LTXV_LORA_COMFY_RENAMING_MAP
    ),
]

# Initialize pipeline
pipeline = DistilledPipeline(
    checkpoint_path=CHECKPOINT_PATH,
    distilled_lora=distilled_lora,
    spatial_upsampler_path=UPSAMPLER_PATH,
    gemma_root=GEMMA_PATH,
    loras=[],
    fp8transformer=False,  # Already FP8 checkpoint
)

print("\n✅ Pipeline loaded!")
print(f"   GPU Memory used: {torch.cuda.memory_allocated() / 1024**3:.1f} GB")

## 5️⃣ Generate Video

### Text-to-Video

In [ ]:
from ltx_pipelines.utils.media_io import encode_video
from ltx_pipelines.utils.constants import AUDIO_SAMPLE_RATE
from ltx_core.model.video_vae import TilingConfig, get_video_chunks_number
from IPython.display import Video, display

# ============================================
# 🎬 EDIT YOUR PROMPT HERE
# ============================================
PROMPT = "A golden retriever running through a sunny meadow, slow motion, cinematic lighting, high quality"
NEGATIVE_PROMPT = "blurry, low quality, distorted, ugly"

# Video settings
SEED = 42              # Change for different results
HEIGHT = 544           # 544 for 16:9, or 768 for square
WIDTH = 960            # 960 for 16:9, or 768 for square
NUM_FRAMES = 49        # 49 = ~2 seconds at 25fps (must be 1 + n*8)
FRAME_RATE = 25.0
# ============================================

print(f"🎬 Generating video...")
print(f"   Prompt: {PROMPT[:60]}...")
print(f"   Resolution: {WIDTH}x{HEIGHT}")
print(f"   Frames: {NUM_FRAMES} ({NUM_FRAMES/FRAME_RATE:.1f}s)")
print(f"   Seed: {SEED}")
print("\n   This takes 1-3 minutes...\n")

tiling_config = TilingConfig.default()
video_chunks_number = get_video_chunks_number(NUM_FRAMES, tiling_config)

# Generate
video, audio = pipeline(
    prompt=PROMPT,
    negative_prompt=NEGATIVE_PROMPT,
    seed=SEED,
    height=HEIGHT,
    width=WIDTH,
    num_frames=NUM_FRAMES,
    frame_rate=FRAME_RATE,
    images=[],
    tiling_config=tiling_config,
    enhance_prompt=False,
)

# Save video
OUTPUT_PATH = "/content/output.mp4"
encode_video(
    video=video,
    fps=FRAME_RATE,
    audio=audio,
    audio_sample_rate=AUDIO_SAMPLE_RATE,
    output_path=OUTPUT_PATH,
    video_chunks_number=video_chunks_number,
)

print(f"\n✅ Video saved to {OUTPUT_PATH}")

# Display video
display(Video(OUTPUT_PATH, embed=True, width=640))

### Image-to-Video (I2V)

Upload an image and animate it!

In [ ]:
from google.colab import files
from PIL import Image
import os

# Upload image
print("📤 Upload an image to animate:")
uploaded = files.upload()

if uploaded:
    INPUT_IMAGE = list(uploaded.keys())[0]
    print(f"\n✅ Uploaded: {INPUT_IMAGE}")
    
    # Show uploaded image
    img = Image.open(INPUT_IMAGE)
    display(img.resize((400, int(400 * img.height / img.width))))
else:
    print("❌ No image uploaded")

In [ ]:
# Generate video from uploaded image
from ltx_pipelines.utils.media_io import encode_video
from ltx_pipelines.utils.constants import AUDIO_SAMPLE_RATE
from ltx_core.model.video_vae import TilingConfig, get_video_chunks_number
from IPython.display import Video, display

# ============================================
# 🎬 EDIT YOUR SETTINGS HERE
# ============================================
PROMPT = "The scene comes to life with gentle movement, cinematic"
NEGATIVE_PROMPT = "blurry, low quality, static, frozen"
SEED = 42
NUM_FRAMES = 49
FRAME_RATE = 25.0

# Image settings
IMAGE_FRAME_INDEX = 0   # Which frame to place the image (0 = first frame)
IMAGE_STRENGTH = 1.0    # How strongly to condition on the image (0-1)
# ============================================

# Get image dimensions and adjust to valid sizes
img = Image.open(INPUT_IMAGE)
width, height = img.size

# Round to nearest multiple of 32
HEIGHT = (min(height, 544) // 32) * 32
WIDTH = (min(width, 960) // 32) * 32

print(f"🎬 Generating I2V...")
print(f"   Input: {INPUT_IMAGE}")
print(f"   Prompt: {PROMPT[:60]}...")
print(f"   Resolution: {WIDTH}x{HEIGHT}")
print("\n   This takes 1-3 minutes...\n")

tiling_config = TilingConfig.default()
video_chunks_number = get_video_chunks_number(NUM_FRAMES, tiling_config)

# Image conditioning
images = [(INPUT_IMAGE, IMAGE_FRAME_INDEX, IMAGE_STRENGTH)]

# Generate
video, audio = pipeline(
    prompt=PROMPT,
    negative_prompt=NEGATIVE_PROMPT,
    seed=SEED,
    height=HEIGHT,
    width=WIDTH,
    num_frames=NUM_FRAMES,
    frame_rate=FRAME_RATE,
    images=images,
    tiling_config=tiling_config,
    enhance_prompt=False,
)

# Save
OUTPUT_PATH = "/content/output_i2v.mp4"
encode_video(
    video=video,
    fps=FRAME_RATE,
    audio=audio,
    audio_sample_rate=AUDIO_SAMPLE_RATE,
    output_path=OUTPUT_PATH,
    video_chunks_number=video_chunks_number,
)

print(f"\n✅ Video saved!")
display(Video(OUTPUT_PATH, embed=True, width=640))

## 6️⃣ Download Video

In [ ]:
from google.colab import files

# Download the generated video
print("📥 Downloading video...")
files.download("/content/output.mp4")

---

## 📝 Tips

**Prompting:**
- Be detailed and specific
- Describe movements, camera angles, lighting
- Keep prompts under 200 words

**Memory Issues:**
- Reduce `NUM_FRAMES` (try 25 instead of 49)
- Reduce resolution (try 512x512)
- Restart runtime and try again

**Quality:**
- Higher frame counts = smoother motion
- Try different seeds for variety
- Use negative prompts to avoid artifacts

---

**Links:**
- [LTX-2 GitHub](https://github.com/Lightricks/LTX-2)
- [Prompting Guide](https://ltx.video/blog/how-to-prompt-for-ltx-2)